# Demo for using `hilucsi4py` with user `Options` #
In this example, we show how to use `hilucs4py` MILU preconditioner coupling with the built-in FGMRES solver.. The example system is a saddle-point formulation of 3D Stokes equation with Taylor-Hood elements.

In [1]:
from scipy.io import loadmat
from hilucsi4py import *
import numpy as np

The matrix is stored by the HILUCSI native binary format that is leading symmetric block aware. It's worht noting that. The following code shows how to load the matrix.

In [2]:
# load the MATFILE from scipy.io
f = loadmat('../demo_inputs/matlab.mat')
A = f['A']
b = f['b'].reshape(-1)

Let's show some basic information of the system, including shape, nnz, and leading block symmetry

In [3]:
# A is scipy.sparse.csr_matrix
print('The system shape is {}, where the nnz is {}'.format(A.shape, A.nnz))

The system shape is (2990, 2990), where the nnz is 44632


Now, let's build the preconditioenr $\boldsymbol{M}$ with more aggressive options, i.e. `droptol` for L and U factors is 1e-2, `condest` for L, U, and D is 5, and $\alpha$ for L and U is 3.

In [4]:
M = HILUCSI()
opts = Options()
opts['tau_L'] = opts['tau_U'] = 0.01
opts['tau_kappa'] = opts['tau_d'] = 5.0
opts['alpha_L'] = opts['alpha_U'] = 3
M.factorize(A, opts=opts)


|    Hierarchical ILU Crout with Scalability and Inverse Thresholds   |
|                                                                     |
| HILUCSI is a package for computing multilevel incomplete LU factor- |
| ization with nearly linear time complexity. In addition, HILUCSI    |
| can also be very robust.                                            |
-----------------------------------------------------------------------

 Package information:

		Copyright (C) The HILUCSI AUTHORS
		Version: 1.0.0
		Built on: 20:05:09, Jul 15 2019


Options (control parameters) are:

tau_L                         0.010000
tau_U                         0.010000
tau_d                         5.000000
tau_kappa                     5.000000
alpha_L                       3
alpha_U                       3
rho                           0.500000
c_d                           10.000000
c_h                           2.000000
N                             -1
verbose                       info
rf_par       

With the preconditioenr successfully been built, let's print out some basic information

In [6]:
print('M levels are {}, with nnz {}'.format(M.levels, M.nnz))

M levels are 2, with nnz 90664


Now, we solve with the built-in flexible GMRES solver, with default configurations, i.e. restart is 30, relative convergence tolerance is 1e-6, and maximum allowed iterations are 500.

In [7]:
solver = FGMRES(M)

In [8]:
x, iters = solver.solve(A, b)

- FGMRES -
rtol=1e-06
restart=30
maxiter=500
kernel: tradition
init-guess: no
trunc: no

Calling traditional GMRES kernel...
Enter outer iteration 1...
  At iteration 1 (inner:1), relative residual is 0.000216346.
  At iteration 2 (inner:1), relative residual is 5.54575e-06.
  At iteration 3 (inner:1), relative residual is 4.62893e-07.


In [9]:
print('solver done, with {} iterations and residule is {}'.format(iters, solver.resids[-1]))

solver done, with 3 iterations and residule is 4.6289339210757907e-07
